In [1]:
import duckdb as ddb
import pandas as pd
import geopandas as gpd

ddb.load_extension('spatial')


In [2]:
optionA = ddb.sql('SELECT time_of_trip,start_lat,start_lng FROM "NYC/other-Lyft_B02510.csv"').df()

In [14]:
ddb.sql('CREATE TABLE nyc AS SELECT time_of_trip,start_lat,start_lng FROM "NYC/other-Lyft_B02510.csv"')

In [3]:
type(optionA)

pandas.core.frame.DataFrame

In [13]:
# Drop the nyc table if it exists
ddb.sql('DROP TABLE IF EXISTS nyc')

In [4]:
optionA

,time_of_trip,start_lat,start_lng
0,9/4/2014 9:51,40.64705,-73.77988
1,8/27/2014 21:13,40.74916,-73.98373
2,9/4/2014 14:16,40.64065,-73.97594
3,9/4/2014 16:08,40.75002,-73.99514
4,8/28/2014 2:41,40.76715,-73.98636
...,...,...,...
267696,9/7/2014 3:19,40.71931,-73.99138
267697,9/7/2014 2:59,40.73868,-73.98862
267698,9/7/2014 1:55,40.71554,-73.94877
267699,9/5/2014 23:26,40.74667,-73.97445


In [5]:
gdf = gpd.GeoDataFrame(optionA, geometry=gpd.points_from_xy(optionA.start_lng, optionA.start_lat))

In [6]:
gdf

,time_of_trip,start_lat,start_lng,geometry
0,9/4/2014 9:51,40.64705,-73.77988,POINT (-73.77988 40.64705)
1,8/27/2014 21:13,40.74916,-73.98373,POINT (-73.98373 40.74916)
2,9/4/2014 14:16,40.64065,-73.97594,POINT (-73.97594 40.64065)
3,9/4/2014 16:08,40.75002,-73.99514,POINT (-73.99514 40.75002)
4,8/28/2014 2:41,40.76715,-73.98636,POINT (-73.98636 40.76715)
...,...,...,...,...
267696,9/7/2014 3:19,40.71931,-73.99138,POINT (-73.99138 40.71931)
267697,9/7/2014 2:59,40.73868,-73.98862,POINT (-73.98862 40.73868)
267698,9/7/2014 1:55,40.71554,-73.94877,POINT (-73.94877 40.71554)
267699,9/5/2014 23:26,40.74667,-73.97445,POINT (-73.97445 40.74667)


In [7]:
# Export the GeoDataFrame to parquet
gdf.to_parquet('nyc.parquet')

In [11]:
parquet = ddb.read_parquet('nyc.parquet')

In [14]:
parquet

┌─────────────────┬───────────┬───────────┬────────────────────────────────────────────────────────────────────┐
│  time_of_trip   │ start_lat │ start_lng │                              geometry                              │
│     varchar     │  double   │  double   │                                blob                                │
├─────────────────┼───────────┼───────────┼────────────────────────────────────────────────────────────────────┤
│ 9/4/2014 9:51   │  40.64705 │ -73.77988 │ \x01\x01\x00\x00\x00}\xB3\xCD\x8D\xE9qR\xC0;p\xCE\x88\xD2RD@       │
│ 8/27/2014 21:13 │  40.74916 │ -73.98373 │ \x01\x01\x00\x00\x00\x05\x86\xACn\xF5~R\xC0V\xBC\x91y\xE4_D@       │
│ 9/4/2014 14:16  │  40.64065 │ -73.97594 │ \x01\x01\x00\x00\x00\xED\xB6\x0B\xCDu~R\xC0Y\x17\xB7\xD1\x00RD@    │
│ 9/4/2014 16:08  │  40.75002 │ -73.99514 │ \x01\x01\x00\x00\x00A\xBC\xAE_\xB0\x7FR\xC0G\xAC\xC5\xA7\x00`D@    │
│ 8/28/2014 2:41  │  40.76715 │ -73.98636 │ \x01\x01\x00\x00\x00I\x85\xB1\x85 \x7FR\xC0.\x90\xA0

In [26]:
ddb.sql("DROP TABLE IF EXISTS nyc_geoparquet")

In [27]:
ddb.sql(f"CREATE TABLE nyc_geoparquet AS SELECT * EXCLUDE geometry, ST_GEOMFROMWKB(geometry) AS geometry FROM 'parquet'")
# CREATE TABLE attapeu AS SELECT * EXCLUDE geometry, ST_GEOMFROMWKB(geometry) AS geometry FROM 
#'s3://us-west-2.opendata.source.coop/google-research-open-buildings/geoparquet-admin1/country=LAO/Attapeu.parquet';

In [36]:
ddb.sql('SELECT * FROM nyc_geoparquet WHERE ST_Y(geometry) > 40')

┌─────────────────┬───────────┬───────────┬────────────────────────────┐
│  time_of_trip   │ start_lat │ start_lng │          geometry          │
│     varchar     │  double   │  double   │          geometry          │
├─────────────────┼───────────┼───────────┼────────────────────────────┤
│ 9/4/2014 9:51   │  40.64705 │ -73.77988 │ POINT (-73.77988 40.64705) │
│ 8/27/2014 21:13 │  40.74916 │ -73.98373 │ POINT (-73.98373 40.74916) │
│ 9/4/2014 14:16  │  40.64065 │ -73.97594 │ POINT (-73.97594 40.64065) │
│ 9/4/2014 16:08  │  40.75002 │ -73.99514 │ POINT (-73.99514 40.75002) │
│ 8/28/2014 2:41  │  40.76715 │ -73.98636 │ POINT (-73.98636 40.76715) │
│ 9/13/2014 3:57  │  40.70707 │ -74.01211 │ POINT (-74.01211 40.70707) │
│ 9/11/2014 23:13 │  40.72953 │ -73.97799 │ POINT (-73.97799 40.72953) │
│ 8/27/2014 22:56 │  40.76689 │ -73.95631 │ POINT (-73.95631 40.76689) │
│ 9/10/2014 17:48 │  40.82332 │ -73.86235 │ POINT (-73.86235 40.82332) │
│ 9/12/2014 23:24 │  40.72188 │ -73.99732 │ POINT (

In [77]:
# Get the max latitude, min latitude, max longitude, and min longitude
max_lat = ddb.sql('SELECT MAX(ST_Y(geometry)) FROM nyc_geoparquet').df()
min_lat = ddb.sql('SELECT MIN(ST_Y(geometry)) FROM nyc_geoparquet').df()
max_lon = ddb.sql('SELECT MAX(ST_X(geometry)) FROM nyc_geoparquet').df()
min_lon = ddb.sql('SELECT MIN(ST_X(geometry)) FROM nyc_geoparquet').df()


In [78]:
min_lat

,min(st_y(geometry))
0,40.52785


In [79]:
# Add 0.1 to the min latitude
min_lat = min_lat + 0.1
min_lat

,min(st_y(geometry))
0,40.62785


In [75]:
max_lat

,max(st_y(geometry))
0,40.93562


In [80]:
# Subtract 0.1 from the max latitude
max_lat = max_lat - 0.1
max_lat

,max(st_y(geometry))
0,40.83562


In [91]:

# Use ST_GeomFromText to create a bounding box polygon
#bbox = ddb.sql(f"SELECT ST_GeomFromText('POLYGON(({min_lon.iloc[0,0]} {min_lat.iloc[0,0]}, {min_lon.iloc[0,0]} {max_lat.iloc[0,0]}, {max_lon.iloc[0,0]} {max_lat.iloc[0,0]}, {max_lon.iloc[0,0]} {min_lat.iloc[0,0]}, {min_lon.iloc[0,0]} {min_lat.iloc[0,0]})))')")
bbox = ddb.sql(f"SELECT ST_GeomFromText('POLYGON(({min_lon.iloc[0,0]} {min_lat.iloc[0,0]}, {min_lon.iloc[0,0]} {max_lat.iloc[0,0]}, {max_lon.iloc[0,0]} {max_lat.iloc[0,0]}, {max_lon.iloc[0,0]} {min_lat.iloc[0,0]}, {min_lon.iloc[0,0]} {min_lat.iloc[0,0]})))') AS geometry")

In [97]:
# Create table AOI and add bbox
ddb.sql('DROP TABLE IF EXISTS AOI')

ddb.sql('CREATE TABLE AOI AS SELECT * FROM bbox')
ddb.table('AOI')





┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                    geometry                                                    │
│                                                    geometry                                                    │
├────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ POLYGON ((-74.22689 40.62785, -74.22689 40.83562, -73.72315 40.83562, -73.72315 40.62785, -74.22689 40.62785)) │
└────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [99]:
# Select all points that are within the bounding box
ddb.sql('SELECT * FROM nyc_geoparquet WHERE ST_Contains((SELECT geometry FROM AOI), geometry)').df()

,time_of_trip,start_lat,start_lng,geometry
0,9/4/2014 9:51,40.64705,-73.77988,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
1,8/27/2014 21:13,40.74916,-73.98373,"[0, 0, 24, 0, 58, 52, 51, 56, 0, 0, 0, 0, 1, 0..."
2,9/4/2014 14:16,40.64065,-73.97594,"[0, 0, 24, 0, 47, 49, 50, 47, 0, 0, 0, 0, 1, 0..."
3,9/4/2014 16:08,40.75002,-73.99514,"[0, 0, 24, 0, 56, 47, 50, 48, 0, 0, 0, 0, 1, 0..."
4,8/28/2014 2:41,40.76715,-73.98636,"[0, 0, 24, 0, 48, 49, 52, 32, 0, 0, 0, 0, 1, 0..."
...,...,...,...,...
258107,9/7/2014 3:19,40.71931,-73.99138,"[0, 0, 24, 0, 116, 147, 24, 126, 0, 0, 0, 0, 1..."
258108,9/7/2014 2:59,40.73868,-73.98862,"[0, 0, 24, 0, 94, 68, 64, 1, 0, 0, 0, 0, 1, 0,..."
258109,9/7/2014 1:55,40.71554,-73.94877,"[0, 0, 24, 0, 72, 162, 127, 82, 0, 0, 0, 0, 1,..."
258110,9/5/2014 23:26,40.74667,-73.97445,"[0, 0, 24, 0, 68, 64, 1, 1, 0, 0, 0, 0, 1, 0, ..."
